# TruBot

In [1]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import sys

sys.path.append(str(Path().cwd().parent.parent.resolve()))

In [2]:
from IPython.display import JSON
from langchain.chains import ConversationalRetrievalChain
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.memory import ConversationSummaryBufferMemory
from langchain.vectorstores import DocArrayHnswSearch
import numpy as np

from trulens_eval import Tru
from trulens_eval import tru_feedback
from trulens_eval.keys import *
from trulens_eval.tru_db import Query
from trulens_eval.tru_db import Record
from trulens_eval.tru_feedback import Huggingface
from trulens_eval.utils.langchain import WithFilterDocuments

# if using Pinecone vectordb:
# from langchain.vectorstores import Pinecone
# import pinecone

KEY SET: OPENAI_API_KEY
KEY SET: PINECONE_API_KEY
KEY SET: PINECONE_ENV
KEY SET: HUGGINGFACE_API_KEY
KEY SET: SLACK_TOKEN
KEY SET: SLACK_SIGNING_SECRET
KEY SET: COHERE_API_KEY


In [3]:
# Set up GPT-3 model

model_name = "gpt-3.5-turbo"

chain_id = "TruBot"

identity = lambda h: h

# Embedding needed for Pinecone vector db.
embedding = OpenAIEmbeddings(model='text-embedding-ada-002')  # 1536 dims

# Pinecone configuration if using pinecone.
# pinecone.init(
#    api_key=PINECONE_API_KEY,  # find at app.pinecone.io
#    environment=PINECONE_ENV  # next to api key in console
#)
#docsearch = Pinecone.from_existing_index(
#    index_name="llmdemo", embedding=embedding
#)

# Pinecone alternative. Requires precomputed 'hnswlib_truera' folder.
docsearch = DocArrayHnswSearch.from_params(
    embedding=embedding,
    work_dir='hnswlib_truera',
    n_dim=1536,
    max_elements=1024
)

retriever = docsearch.as_retriever()

# LLM for completing prompts, and other tasks.
llm = OpenAI(temperature=0, max_tokens=128)

# Conversation memory.
memory = ConversationSummaryBufferMemory(
    max_token_limit=650,
    llm=llm,
    memory_key="chat_history",
    output_key='answer'
)

# Language mismatch fix:
"""
chain.combine_docs_chain.llm_chain.prompt.template = \
    "Use the following pieces of context to answer the question at the end " \
    "in the same language as the question. If you don't know the answer, " \
    "just say that you don't know, don't try to make up an answer.\n\n" \
    "{context}\n\n" \
    "Question: {question}\n" \
    "Helpful Answer: "
"""

# Poor contexts fix using prompts:
"""
chain.combine_docs_chain.llm_chain.prompt.template = \
    "Use only the relevant contexts to answer the question at the end " \
    ". Some pieces of context may not be relevant. If you don't know the answer, " \
    "just say that you don't know, don't try to make up an answer.\n\n" \
    "Contexts: \n{context}\n\n" \
    "Question: {question}\n" \
    "Helpful Answer: "
chain.combine_docs_chain.document_prompt.template="\tContext: {page_content}"
"""

# Better contexts fix, filter contexts with relevance:
"""
def filter_by_relevance(query, doc):
    return openai.qs_relevance(question=query, statement=doc.page_content) > 0.5

retriever = WithFilterDocuments.of_retriever(
    retriever=retriever, filter_func=filter_by_relevance
)
"""

# Conversational chain puts it all together.
chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    return_source_documents=True,
    memory=memory,
    get_chat_history=identity,
    max_tokens_limit=4096
)

In [4]:
# Feedbacl functions.

hugs = tru_feedback.Huggingface()
openai = tru_feedback.OpenAI()

f_toxic = tru_feedback.Feedback(hugs.not_toxic).on_response()
f_lang_match = tru_feedback.Feedback(hugs.language_match).on(
    text1="prompt", text2="response"
)
f_relevance = tru_feedback.Feedback(openai.relevance).on(
    prompt="input", response="output"
)
f_qs_relevance = tru_feedback.Feedback(openai.qs_relevance).on(
    question="input",
    statement=Record.chain.combine_docs_chain._call.args.inputs.input_documents
).on_multiple(
    multiarg="statement", each_query=Record.page_content, agg=np.min
)

feedbacks=[f_toxic, f_lang_match, f_relevance, f_qs_relevance]

huggingface api: 0requests [00:00, ?requests/s]

openai api: 0requests [00:00, ?requests/s]

In [6]:
# Trulens instrumentation.

tc = Tru().Chain(
    chain,
    chain_id=chain_id,
    feedbacks=feedbacks,
)

✅ chain TruBot -> default.sqlite
✅ feedback def. feedback_hash_c479553ae6ae61dc58f9b9c1d609ca40 -> default.sqlite
✅ feedback def. feedback_hash_26c84c76d907d808c36028e024af63c0 -> default.sqlite
✅ feedback def. feedback_hash_90b443be1347d732ededd19cd3968c72 -> default.sqlite
✅ feedback def. feedback_hash_b9475c84f84aee5e117ba7ec7aab7e25 -> default.sqlite


In [7]:
res, record = tc.call_with_record("Who is Shayak?")

In [8]:
JSON(res)

✅ record record_hash_09f84e4ac1efde5fd79ef442a1610be8 from TruBot -> default.sqlite


<IPython.core.display.JSON object>

In [9]:
feedback = Tru().run_feedback_functions(record_json=record, feedback_functions=feedbacks)

✅ feedback feedback_hash_c479553ae6ae61dc58f9b9c1d609ca40 on record_hash_09f84e4ac1efde5fd79ef442a1610be8 -> default.sqlite
✅ feedback feedback_hash_26c84c76d907d808c36028e024af63c0 on record_hash_09f84e4ac1efde5fd79ef442a1610be8 -> default.sqlite
✅ feedback feedback_hash_90b443be1347d732ededd19cd3968c72 on record_hash_09f84e4ac1efde5fd79ef442a1610be8 -> default.sqlite
✅ feedback feedback_hash_b9475c84f84aee5e117ba7ec7aab7e25 on record_hash_09f84e4ac1efde5fd79ef442a1610be8 -> default.sqlite


In [10]:
f_qs_relevance.run_on_record(record_json=record, chain_json=tc.json)

{'_success': True,
 'feedback_id': 'feedback_hash_b9475c84f84aee5e117ba7ec7aab7e25',
 'record_id': 'record_hash_09f84e4ac1efde5fd79ef442a1610be8',
 'qs_relevance': 0.1}